# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [89]:
# Use as many python cells as you wish to write your code
import pandas as pd 

def merge_all_data(df1_path, df2_path, df3_path, df4_path):
    """
    Recebe os caminhos de 4 arquivos CSV, mescla e retorna os dados limpos.
    """
    
    # Leitura dos DataFrames
    df1 = pd.read_csv(df1_path)
    df2 = pd.read_csv(df2_path)
    df3 = pd.read_csv(df3_path)
    df4 = pd.read_csv(df4_path)
    
    df = df4.merge(df1, on='user_id', how='left')
    df = df.merge(df2, on=['user_id', 'date'], how='left')
    df = df.merge(df3, on='experiment_id', how='left')
    
    # Limpeza dos dados
    # User_id - check
    # Date
    df['date'] = pd.to_datetime(df['date'])
    
    # user_age_group (bins)    
    # Criação das categorias
    bins = [0, 17, 25, 35, 45, 55, 65, float('inf')]
    labels = ['Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65']
    df['user_age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
    df['user_age_group'] = df['user_age_group'].cat.add_categories('Unknown')
    df['user_age_group'] = df['user_age_group'].fillna('Unknown') 
    df = df.drop(columns=['age'])
    
    # experiment_name
    df['experiment_name'] = df['name'].astype('category')
    df = df.drop(columns=['name'])
    
    # supplement_name
    df['supplement_name'] = df['supplement_name'].fillna('No intake')
    
    # dosage_grams + dosage_unit
    # No df atual não há unit sem ser mg, mas caso houvesse:
    df['dosage_grams'] = df['dosage']
    df.loc[df['dosage_unit'] == 'mg', 'dosage_grams'] = df['dosage'] / 1000
    df = df.drop(columns=['dosage', 'dosage_unit'])
    
    # is_placebo check
    # average_heart_rate check
    # average_glicose check
    # sleep_hours
    df['sleep_hours'] = df['sleep_hours'].str.extract(r'(\d+\.?\d*)', expand=False).astype(float)
    # Activity level check
    
    # Reordenar as colunas
    col_order = [
    'user_id',
    'date',
    'email',
    'user_age_group',
    'experiment_name',
    'supplement_name',
    'dosage_grams',
    'is_placebo',
    'average_heart_rate',
    'average_glucose',
    'sleep_hours',
    'activity_level'
    ]
    df = df[col_order]
    
    return df

In [90]:
merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-01-31,hi_1@example.com,Over 65,Sleep Quality,Zinc,0.149315,False,93.055612,70.089910,8.8,1
1,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-02-28,hi_1@example.com,Over 65,NaN,No intake,NaN,NaN,88.059964,78.411148,8.0,3
2,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Endurance,Magnesium,0.239949,True,78.373746,107.418818,11.9,1
3,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Focus,Placebo,0.416022,False,78.373746,107.418818,11.9,1
4,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-04-30,hi_1@example.com,Over 65,Recovery,Placebo,0.324913,False,62.204061,117.259092,5.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2716,9c083dc3-b789-4cee-87fb-9887ff4b4e29,2018-04-30,contact499@myemail.com,46-55,NaN,No intake,NaN,NaN,88.575502,121.997505,7.9,3
2717,2f2aa322-d2b2-4c88-aeb6-e49e768521f3,2018-01-31,hello_500@myemail.com,46-55,NaN,No intake,NaN,NaN,96.665724,73.559778,9.9,1
2718,2f2aa322-d2b2-4c88-aeb6-e49e768521f3,2018-02-28,hello_500@myemail.com,46-55,Strength,Vitamin C,0.280467,False,88.219866,85.478628,7.9,4
2719,2f2aa322-d2b2-4c88-aeb6-e49e768521f3,2018-03-31,hello_500@myemail.com,46-55,NaN,No intake,NaN,NaN,86.871171,89.198330,7.9,3
